In [19]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
if os.environ["OPENAI_API_KEY"]:
    print("API Key loaded successfully.")

API Key loaded successfully.


In [21]:
from langchain_openai import ChatOpenAI

In [22]:
llm = ChatOpenAI(model="gpt-5-nano",temperature=0)

## RAG Implemmentation with own text data

In [23]:
# Step1:  Preparing document for text data

from langchain_core.documents import Document

In [24]:
my_text = '''"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).
Part of a series on
Artificial intelligence (AI)

Major goals
Approaches
Applications
Philosophy
History
Controversies
Glossary
vte
Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]

High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general applications, often without being called AI because once something becomes useful enough and common enough it's not labeled AI anymore."[2][3]

Various subfields of AI research are centered around particular goals and the use of particular tools. The traditional goals of AI research include learning, reasoning, knowledge representation, planning, natural language processing, perception, and support for robotics.[a] To reach these goals, AI researchers have adapted and integrated a wide range of techniques, including search and mathematical optimization, formal logic, artificial neural networks, and methods based on statistics, operations research, and economics.[b] AI also draws upon psychology, linguistics, philosophy, neuroscience, and other fields.[4] Some companies, such as OpenAI, Google DeepMind and Meta,[5] aim to create artificial general intelligence (AGI) – AI that can complete virtually any cognitive task at least as well as a human.

Artificial intelligence was founded as an academic discipline in 1956,[6] and the field went through multiple cycles of optimism throughout its history,[7][8] followed by periods of disappointment and loss of funding, known as AI winters.[9][10] Funding and interest vastly increased after 2012 when graphics processing units started being used to accelerate neural networks, and deep learning outperformed previous AI techniques.[11] This growth accelerated further after 2017 with the transformer architecture.[12] In the 2020s, an ongoing period of rapid progress in advanced generative AI became known as the AI boom. Generative AI's ability to create and modify content has led to several unintended consequences and harms. Ethical concerns have been raised about AI's long-term effects and potential existential risks, prompting discussions about regulatory policies to ensure the safety and benefits of the technology.'''

In [25]:
docs = [Document(page_content=my_text, metadata = {"source": "ABC", "documentID": "Doc1"})]

docs

[Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).\nPart of a series on\nArtificial intelligence (AI)\n\nMajor goals\nApproaches\nApplications\nPhilosophy\nHistory\nControversies\nGlossary\nvte\nArtificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]\n\nHigh-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, a

In [26]:
# Step 2: Splitting documents into chunks

In [27]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
chunks = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50
)
chunks = chunks.split_documents(docs)
chunks

[Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).\nPart of a series on\nArtificial intelligence (AI)'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='Major goals\nApproaches\nApplications\nPhilosophy\nHistory\nControversies\nGlossary\nvte'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='Controversies\nGlossary\nvte\nArtificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(metadata={'sour

In [28]:
# Step 3: Create embeddings for the chunks

In [44]:
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model = "text-embedding-3-small")

In [45]:
# Step 4: Create and store Embeddings in vector store


In [46]:
from langchain_community.vectorstores import Chroma  # can also use inmemorey, FAISS, Pinecone, Weaviate etc.

vector_store = Chroma.from_documents(
    documents = chunks,
    embedding = embedding_model
)

In [48]:
vectors = []

for doc in chunks:
    vector = embedding_model.embed_documents([doc.page_content])
    vectors.append(vector)
vectors

[[[-0.014078072272241116,
   -0.027982646599411964,
   -0.007962290197610855,
   0.00028270776965655386,
   0.01651942729949951,
   0.005226608365774155,
   -0.026941662654280663,
   0.012590952217578888,
   -0.02704080380499363,
   0.0299654733389616,
   -0.0046069747768342495,
   -0.0600796602666378,
   0.012312117032706738,
   -0.03933433070778847,
   -0.024463128298521042,
   0.013433652929961681,
   -0.01844029128551483,
   0.008457996882498264,
   0.034005485475063324,
   -0.023583250120282173,
   0.0015847125323489308,
   0.018812071532011032,
   -0.0149827366694808,
   0.03286535665392876,
   0.006165352649986744,
   -0.016383107751607895,
   0.02763565257191658,
   0.03586438298225403,
   -0.015193412080407143,
   0.021885452792048454,
   0.04126758873462677,
   -0.026470741257071495,
   -0.013470831327140331,
   -0.018167652189731598,
   -0.007094803266227245,
   0.014127642847597599,
   -0.004839337430894375,
   -0.028974059969186783,
   0.01680445857346058,
   0.01197131816

In [ ]:
# Step 5: Semantic Search on the vector store

In [53]:
vector_store.similarity_search("What is AI?", k=3)

[Document(metadata={'documentID': 'Doc1', 'source': 'ABC'}, page_content='"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).\nPart of a series on\nArtificial intelligence (AI)'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='Controversies\nGlossary\nvte\nArtificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goals.[1]'),
 Document(metadata={'documentID': 'Doc1', 'source': 'ABC'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (us

In [57]:
vector_store.similarity_search("why Ai is important", k=4)

[Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='"AI" redirects here. For other uses, see AI (disambiguation) and Artificial intelligence (disambiguation).\nPart of a series on\nArtificial intelligence (AI)'),
 Document(metadata={'source': 'ABC', 'documentID': 'Doc1'}, page_content='High-profile applications of AI include advanced web search engines (e.g., Google Search); recommendation systems (used by YouTube, Amazon, and Netflix); virtual assistants (e.g., Google Assistant, Siri, and Alexa); autonomous vehicles (e.g., Waymo); generative and creative tools (e.g., language models and AI art); and superhuman play and analysis in strategy games (e.g., chess and Go). However, many AI applications are not perceived as AI: "A lot of cutting edge AI has filtered into general'),
 Document(metadata={'documentID': 'Doc1', 'source': 'ABC'}, page_content='lot of cutting edge AI has filtered into general applications, often without being called AI because once something b

## Talk to llm

In [58]:
context = vector_store.similarity_search("What is AI?", k=4)

In [61]:
response = llm.invoke(f"What is AI? You can answer using the following context: {context}")
print(response)

content='AI, or artificial intelligence, is the ability of computer systems to perform tasks that typically require human intelligence. These tasks include learning, reasoning, problem-solving, perception, and decision-making. AI is a field of computer science that develops methods and software to enable machines to perceive their environment, learn from data, and take actions to achieve defined goals.\n\nKey points:\n- It spans a wide range of techniques (e.g., neural networks, statistics, optimization) and draws on disciplines like psychology, linguistics, and neuroscience.\n- Real-world AI includes search engines, recommendation systems, virtual assistants, autonomous vehicles, generative tools, and advanced game playing.\n- Some researchers aim for artificial general intelligence (AGI), which would perform virtually any cognitive task as well as or better than humans.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 742, 'prompt_tokens': 